# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'05-14-2021'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'05-14-2021'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-05-15 04:20:58,33.93911,67.709953,63355,2730,54634.0,5991.0,Afghanistan,162.747752,4.309052
1,NaN,NaN,NaN,Albania,2021-05-15 04:20:58,41.15330,20.168300,131939,2427,122105.0,7407.0,Albania,4584.717493,1.839486
2,NaN,NaN,NaN,Algeria,2021-05-15 04:20:58,28.03390,1.659600,125059,3360,87137.0,34562.0,Algeria,285.190480,2.686732
3,NaN,NaN,NaN,Andorra,2021-05-15 04:20:58,42.50630,1.521800,13510,127,13155.0,228.0,Andorra,17485.277940,0.940044
4,NaN,NaN,NaN,Angola,2021-05-15 04:20:58,-11.20270,17.873900,30030,651,25650.0,3729.0,Angola,91.370277,2.167832


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/5/21,5/6/21,5/7/21,5/8/21,5/9/21,5/10/21,5/11/21,5/12/21,5/13/21,5/14/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,60797,61162,61455,61755,61842,62063,62403,62718,63045,63355
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,131419,131510,131577,131666,131723,131753,131803,131845,131890,131939
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,123272,123473,123692,123900,124104,124288,124483,124682,124889,125059


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/5/21,5/6/21,5/7/21,5/8/21,5/9/21,5/10/21,5/11/21,5/12/21,5/13/21,5/14/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,2654,2664,2673,2683,2686,2698,2710,2713,2721,2730
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,2403,2406,2408,2411,2412,2416,2420,2423,2426,2427
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,3299,3307,3315,3321,3328,3335,3343,3350,3355,3360


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/5/21,5/6/21,5/7/21,5/8/21,5/9/21,5/10/21,5/11/21,5/12/21,5/13/21,5/14/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,53750,53961,54019,54040,54222,54382,54503,54534,54619,54634
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,113468,114362,115253,116126,117089,118041,119061,120072,121122,122105
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,85869,86007,86149,86280,86420,86554,86703,86857,87003,87137


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,5/5/21,5/6/21,5/7/21,5/8/21,5/9/21,5/10/21,5/11/21,5/12/21,5/13/21,5/14/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,6914,6914,6918,6918,6920,6920,6926,6928,6938,6971
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,21035,21093,21107,21123,21131,21135,21154,21170,21191,21290
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,2307,2307,2307,2307,2308,2308,2310,2314,2317,2319


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,5/5/21,5/6/21,5/7/21,5/8/21,5/9/21,5/10/21,5/11/21,5/12/21,5/13/21,5/14/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,108,108,108,108,108,108,108,108,108,108
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,307,307,307,308,308,308,308,309,309,309
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,56,57,57,57,57,57,57,56,56,56


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
652,1001.0,Autauga,Alabama,US,2021-05-15 04:20:58,32.539527,-86.644082,6971,108,NaN,NaN,"Autauga, Alabama, US",12477.402495,1.549276
689,1075.0,Lamar,Alabama,US,2021-05-15 04:20:58,33.779950,-88.096680,1430,35,NaN,NaN,"Lamar, Alabama, US",10358.565737,2.447552
690,1077.0,Lauderdale,Alabama,US,2021-05-15 04:20:58,34.901719,-87.656247,9449,243,NaN,NaN,"Lauderdale, Alabama, US",10189.908227,2.571701


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,32894616,585225,0.0
India,24046809,262317,20079599.0
Brazil,15519525,432628,13703280.0


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,32894616,585225,0.0,32309391.0,2021-05-15 04:20:58,37.950547,-91.419547
India,24046809,262317,20079599.0,3704893.0,2021-05-15 04:20:58,23.088275,81.806127
Brazil,15519525,432628,13703280.0,1383617.0,2021-05-15 04:20:58,-12.669522,-48.480493
France,5909386,107584,377338.0,5424464.0,2021-05-15 04:20:58,6.430808,-34.730285
Turkey,5095390,44301,4894024.0,157065.0,2021-05-15 04:20:58,38.963700,35.243300


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,3766499,62626,0.0
Texas,2927195,50903,0.0
Florida,2286203,36000,0.0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,3766499,62626,0.0,3703873.0,2021-05-15 04:20:58,37.843962,-120.728594
Texas,2927195,50903,0.0,2876292.0,2021-05-15 04:20:58,31.660643,-98.653069
Florida,2286203,36000,0.0,2250203.0,2021-05-15 04:20:58,28.940755,-82.700744
New York,2081823,52903,0.0,2028920.0,2021-05-15 04:20:58,42.544151,-75.474183
Illinois,1363409,24725,0.0,1338684.0,2021-05-15 04:20:58,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,1236988,24083,0.0
Illinois,Cook,545785,10114,0.0
Arizona,Maricopa,543928,9947,0.0
Florida,Miami-Dade,493723,6290,0.0
Texas,Harris,396143,6323,0.0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,1236988,24083,0.0,1212905.0,2021-05-15 04:20:58,34.308284,-118.228241,6037.0
Illinois,Cook,545785,10114,0.0,535671.0,2021-05-15 04:20:58,41.841448,-87.816588,17031.0
Arizona,Maricopa,543928,9947,0.0,533981.0,2021-05-15 04:20:58,33.348359,-112.491815,4013.0
Florida,Miami-Dade,493723,6290,0.0,487433.0,2021-05-15 04:20:58,25.611236,-80.551706,12086.0
Texas,Harris,396143,6323,0.0,389820.0,2021-05-15 04:20:58,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-05-12,62718,131845,124682,13470,29405,1240,3215572,219950,29955,633960,...,4457742,228102,95467,4,210948,3658,303270,6498,92211,38466
2021-05-13,63045,131890,124889,13470,29695,1240,3242103,220217,29957,634893,...,4460405,231901,95826,4,211838,3740,303270,6507,92262,38491
2021-05-14,63355,131939,125059,13510,30030,1240,3269466,220447,29964,635780,...,4462603,235206,96117,4,212998,3816,303270,6523,92356,38535


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-05-12,2713,2423,3350,127,645,32,68807,4272,910,10428,...,127901,3252,666,1,2337,35,3401,1277,1259,1579
2021-05-13,2721,2426,3355,127,649,32,69254,4291,910,10444,...,127912,3318,667,1,2352,35,3401,1278,1260,1582
2021-05-14,2730,2427,3360,127,651,32,69853,4301,910,10455,...,127930,3369,667,1,2366,35,3401,1285,1260,1582


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-05-12,54534,120072,86857,13104,25187,1179,2869341,206078,23474,609666,...,15158,198922,90976,3,194166,2636,288891,3003,90574,36277
2021-05-13,54619,121122,87003,13104,25629,1179,2891435,206752,23484,611254,...,15201,201367,91301,3,195380,2657,288891,3003,90635,36286
2021-05-14,54634,122105,87137,13155,25650,1179,2913144,207464,23490,612523,...,15214,204051,91507,3,196759,2657,288891,3008,90699,36308


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,5/5/21,5/6/21,5/7/21,5/8/21,5/9/21,5/10/21,5/11/21,5/12/21,5/13/21,5/14/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,6914,6914,6918,6918,6920,6920,6926,6928,6938,6971
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,1415,1418,1419,1425,1425,1425,1425,1427,1429,1430
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,9407,9413,9419,9420,9424,9428,9427,9437,9441,9449
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,2997,2998,2998,3002,3002,3002,3003,3004,3007,3022
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,15822,15844,15856,15873,15884,15886,15882,15887,15892,15946


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-05-12,531751,69267,0,870624,338043,3763281,526687,344141,106455,49,...,855587,2922680,401251,23651,3256,668147,418020,157215,667637,58893
2021-05-13,532895,69351,0,871168,338235,3764405,528533,344612,106580,49,...,856428,2924766,401669,23724,3261,668726,419382,157626,668208,58996
2021-05-14,537813,69464,0,872022,338485,3766499,530098,344977,106715,49,...,857055,2927195,402005,23792,3267,669219,421757,157923,668805,59079


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2021-05-12        6928   21170    2314  2612   6680    1228   2164   14515   
2021-05-13        6938   21191    2317  2615   6694    1228   2164   14522   
2021-05-14        6971   21290    2319  2630   6750    1230   2178   14556   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-05-12         3575     1852  ...    2741    628     3171      790   
2021-05-13         3578     1847  ...    2752    631     3178      790   
2021-05-14         3581     1843  ...    2758    630     3179      790   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-05-12           4323  3760  2234          0      914    651  
2021-05-13           4337  3762  2237          0      918    651  
2021-05-14           4353  3765  2239          0      918    651  

[3 rows x 3336 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-05-12,10997,353,0,17430,5777,62472,6402,8161,1644,0,...,12296,50785,2249,252,27,10934,5614,2748,7687,712
2021-05-13,11001,353,0,17438,5783,62573,6423,8168,1648,0,...,12305,50840,2255,252,27,10961,5626,2751,7706,712
2021-05-14,11024,353,0,17447,5790,62626,6432,8173,1651,0,...,12312,50903,2256,252,27,10991,5622,2756,7707,712


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-05-12         108     309      56   64    137      41     69     316   
2021-05-13         108     309      56   64    137      41     69     316   
2021-05-14         108     309      56   64    139      41     69     319   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-05-12          123       45  ...      30     11       31        7   
2021-05-13          123       45  ...      30     11       31        7   
2021-05-14          123       45  ...      30     11       31        7   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-05-12             38     9    13          0       26      5  
2021-05-13             38     9    13          0       26      5  
2021-05-14             38     9    13          0       26      5  

[3 rows x 3336 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-05-12,0.005048,0.000319,0.001599,0.00171,0.008886,0.001616,0.007670,0.001612,0.000301,0.001887,...,0.000515,0.009985,0.004155,0.0,0.003960,0.024363,0.001628,0.000924,0.000640,0.000468
2021-05-13,0.005214,0.000341,0.001660,0.00000,0.009862,0.000000,0.008251,0.001214,0.000067,0.001472,...,0.000597,0.016655,0.003760,0.0,0.004219,0.022417,0.000000,0.001385,0.000553,0.000650
2021-05-14,0.004917,0.000372,0.001361,0.00297,0.011281,0.000000,0.008440,0.001044,0.000234,0.001397,...,0.000493,0.014252,0.003037,0.0,0.005476,0.020321,0.000000,0.002459,0.001019,0.001143


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-05-12,0.001107,0.001240,0.002094,0.0,0.009390,0.0,0.007261,0.003759,0.0,0.001441,...,0.000086,0.013716,0.003012,0.0,0.007328,0.0,0.002358,0.000784,0.000795,0.0000
2021-05-13,0.002949,0.001238,0.001493,0.0,0.006202,0.0,0.006496,0.004448,0.0,0.001534,...,0.000086,0.020295,0.001502,0.0,0.006418,0.0,0.000000,0.000783,0.000794,0.0019
2021-05-14,0.003308,0.000412,0.001490,0.0,0.003082,0.0,0.008649,0.002330,0.0,0.001053,...,0.000141,0.015371,0.000000,0.0,0.005952,0.0,0.000000,0.005477,0.000000,0.0000


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-05-12,0.000569,0.008491,0.001776,0.002601,0.001670,0.000849,0.005247,0.001959,0.000341,0.003215,...,0.000726,0.009321,0.003441,0.0,0.005578,0.006875,0.003331,0.000000,0.000387,0.001546
2021-05-13,0.001559,0.008745,0.001681,0.000000,0.017549,0.000000,0.007700,0.003271,0.000426,0.002605,...,0.002837,0.012291,0.003572,0.0,0.006252,0.007967,0.000000,0.000000,0.000673,0.000248
2021-05-14,0.000275,0.008116,0.001540,0.003892,0.000819,0.000000,0.007508,0.003444,0.000255,0.002076,...,0.000855,0.013329,0.002256,0.0,0.007058,0.000000,0.000000,0.001665,0.000706,0.000606


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-05-12,0.000653,0.001287,NaN,0.000539,0.000663,0.000399,0.002308,0.000544,0.001072,0.0,...,0.000783,0.000903,0.001168,0.001906,0.007114,0.000840,0.002614,0.002167,0.000994,0.000986
2021-05-13,0.002151,0.001213,NaN,0.000625,0.000568,0.000299,0.003505,0.001369,0.001174,0.0,...,0.000983,0.000714,0.001042,0.003087,0.001536,0.000867,0.003258,0.002614,0.000855,0.001749
2021-05-14,0.009229,0.001629,NaN,0.000980,0.000739,0.000556,0.002961,0.001059,0.001267,0.0,...,0.000732,0.000830,0.000837,0.002866,0.001840,0.000737,0.005663,0.001884,0.000893,0.001407


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-05-12      0.000289  0.000756  0.001732  0.001150  0.000299  0.000000   
2021-05-13      0.001443  0.000992  0.001296  0.001149  0.002096  0.000000   
2021-05-14      0.004756  0.004672  0.000863  0.005736  0.008366  0.001629   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-05-12      0.001852  0.000276  0.000560  0.000000  ... -0.000365   
2021-05-13      0.000000  0.000482  0.000839 -0.002700  ...  0.004013   
2021-05-14      0.006470  0.002341  0.000838 -0.002166  ...  0.002180   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-05-12      0.004800  0.001263  0.001267   0.002319  0.000532  0.000896   
2021-05-13      0.004777  0.002208  0.000000   0.003238  0.000532  0.001343   
2021-05-14     -0.001585  0.000315  0.000000   0.003689  0.000797  0.000894   

Province_State                              
Admin2         Unassigned  Washakie Weston  
2021-05-12            NaN  0.001095    0.0  
2021-05-13            NaN  0.004376    0.0  
2021-05-14            NaN  0.000000    0.0  

[3 rows x 3336 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-05-12,0.001092,0.005698,NaN,0.000115,0.001213,0.001716,0.004866,0.000613,0.000000,NaN,...,0.000325,0.001301,0.005814,0.003984,0.0,0.001374,0.003755,0.005121,0.002739,0.0
2021-05-13,0.000364,0.000000,NaN,0.000459,0.001039,0.001617,0.003280,0.000858,0.002433,NaN,...,0.000732,0.001083,0.002668,0.000000,0.0,0.002469,0.002138,0.001092,0.002472,0.0
2021-05-14,0.002091,0.000000,NaN,0.000516,0.001210,0.000847,0.001401,0.000612,0.001820,NaN,...,0.000569,0.001239,0.000443,0.000000,0.0,0.002737,-0.000711,0.001818,0.000130,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                    \
Admin2         Autauga   Baldwin   Barbour Bibb    Blount Bullock Butler   
2021-05-12         0.0  0.003247 -0.017544  0.0  0.000000     0.0    0.0   
2021-05-13         0.0  0.000000  0.000000  0.0  0.000000     0.0    0.0   
2021-05-14         0.0  0.000000  0.000000  0.0  0.014599     0.0    0.0   

Province_State                              ... Wyoming                  \
Admin2           Calhoun Chambers Cherokee  ...    Park Platte Sheridan   
2021-05-12      0.006369      0.0      0.0  ...     0.0    0.0      0.0   
2021-05-13      0.000000      0.0      0.0  ...     0.0    0.0      0.0   
2021-05-14      0.009494      0.0      0.0  ...     0.0    0.0      0.0   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-05-12          0.0        0.0   0.0   0.0        NaN      0.0    0.0  
2021-05-13          0.0        0.0   0.0   0.0        NaN      0.0    0.0  
2021-05-14          0.0        0.0   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3336 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-05-12,0.004732,0.000349,0.001589,0.001421,0.008585,0.001568,0.007222,0.001295,0.000295,0.001681,...,0.000520,0.010923,0.003929,7.947286e-08,0.004682,0.025629,0.001639,0.001153,0.000520,0.000410
2021-05-13,0.004973,0.000345,0.001625,0.000711,0.009224,0.000784,0.007736,0.001255,0.000181,0.001576,...,0.000559,0.013789,0.003845,3.973643e-08,0.004450,0.024023,0.000820,0.001269,0.000537,0.000530
2021-05-14,0.004945,0.000358,0.001493,0.001840,0.010253,0.000392,0.008088,0.001150,0.000207,0.001487,...,0.000526,0.014020,0.003441,1.986821e-08,0.004963,0.022172,0.000410,0.001864,0.000778,0.000837


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-05-12,0.002510,0.001334,0.002177,0.000008,0.007148,2.461427e-07,0.006956,0.003102,5.122786e-13,0.001524,...,0.000093,0.013693,0.002461,0.0,0.006906,5.495406e-78,0.003404,0.001208,0.000633,0.000495
2021-05-13,0.002730,0.001286,0.001835,0.000004,0.006675,1.230713e-07,0.006726,0.003775,2.561393e-13,0.001529,...,0.000089,0.016994,0.001981,0.0,0.006662,2.747703e-78,0.001702,0.000996,0.000714,0.001197
2021-05-14,0.003019,0.000849,0.001663,0.000002,0.004878,6.153567e-08,0.007688,0.003053,1.280697e-13,0.001291,...,0.000115,0.016182,0.000991,0.0,0.006307,1.373851e-78,0.000851,0.003236,0.000357,0.000599


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-05-12,0.001497,0.008419,0.001715,0.002797,0.005097,0.01184,0.005918,0.002333,0.000429,0.003282,...,0.002156,0.010905,0.003582,1.907349e-06,0.005989,0.004727,0.003266,0.000915,0.000479,0.001459
2021-05-13,0.001528,0.008582,0.001698,0.001399,0.011323,0.00592,0.006809,0.002802,0.000427,0.002943,...,0.002497,0.011598,0.003577,9.536743e-07,0.006121,0.006347,0.001633,0.000458,0.000576,0.000854
2021-05-14,0.000901,0.008349,0.001619,0.002645,0.006071,0.00296,0.007159,0.003123,0.000341,0.002510,...,0.001676,0.012464,0.002917,4.768372e-07,0.006589,0.003173,0.000817,0.001061,0.000641,0.000730


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-05-12,0.000574,0.001347,NaN,0.000651,0.000593,0.000417,0.002270,0.000921,0.001094,3.143142e-128,...,0.000810,0.000878,0.000883,0.002214,0.005040,0.000837,0.002781,0.001963,0.000850,0.001143
2021-05-13,0.001363,0.001280,NaN,0.000638,0.000581,0.000358,0.002888,0.001145,0.001134,1.571571e-128,...,0.000897,0.000796,0.000962,0.002650,0.003288,0.000852,0.003019,0.002289,0.000853,0.001446
2021-05-14,0.005296,0.001455,NaN,0.000809,0.000660,0.000457,0.002924,0.001102,0.001200,7.857854e-129,...,0.000814,0.000813,0.000899,0.002758,0.002564,0.000794,0.004341,0.002086,0.000873,0.001426


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-05-12      0.000391  0.000715  0.001118  0.000838  0.000903 -0.000400   
2021-05-13      0.000917  0.000854  0.001207  0.000993  0.001499 -0.000200   
2021-05-14      0.002837  0.002763  0.001035  0.003365  0.004932  0.000714   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-05-12      0.001057  0.000333  0.001063  0.000599  ...  0.000761   
2021-05-13      0.000528  0.000407  0.000951 -0.001050  ...  0.002387   
2021-05-14      0.003499  0.001374  0.000895 -0.001608  ...  0.002284   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-05-12      0.002813  0.001200  0.002256   0.002250  0.000616  0.001533   
2021-05-13      0.003795  0.001704  0.001128   0.002744  0.000574  0.001438   
2021-05-14      0.001105  0.001009  0.000564   0.003217  0.000686  0.001166   

Province_State                                     
Admin2         Unassigned  Washakie        Weston  
2021-05-12           -1.0  0.000984  1.645124e-06  
2021-05-13           -1.0  0.002680  8.225620e-07  
2021-05-14           -1.0  0.001340  4.112810e-07  

[3 rows x 3336 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-05-12,0.000786,0.004307,NaN,0.000394,0.000962,0.001120,0.003061,0.000654,0.000983,NaN,...,0.000733,0.001054,0.004271,0.004037,2.292486e-09,0.001290,0.002753,0.003313,0.001991,0.000713
2021-05-13,0.000575,0.002154,NaN,0.000426,0.001000,0.001368,0.003170,0.000756,0.001708,NaN,...,0.000732,0.001069,0.003470,0.002018,1.146243e-09,0.001880,0.002445,0.002202,0.002231,0.000356
2021-05-14,0.001333,0.001077,NaN,0.000471,0.001105,0.001108,0.002286,0.000684,0.001764,NaN,...,0.000651,0.001154,0.001957,0.001009,5.731216e-10,0.002308,0.000867,0.002010,0.001181,0.000178


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-05-12      0.000037  0.001739 -0.008632  0.000249  0.000144  0.000006   
2021-05-13      0.000018  0.000869 -0.004316  0.000124  0.000072  0.000003   
2021-05-14      0.000009  0.000435 -0.002158  0.000062  0.007335  0.000002   

Province_State                                                      ...  \
Admin2                Butler   Calhoun      Chambers      Cherokee  ...   
2021-05-12      2.829987e-08  0.003247  3.006090e-06  2.712009e-09  ...   
2021-05-13      1.414994e-08  0.001624  1.503045e-06  1.356005e-09  ...   
2021-05-14      7.074968e-09  0.005559  7.515225e-07  6.780024e-10  ...   

Province_State       Wyoming                                            \
Admin2                  Park        Platte      Sheridan      Sublette   
2021-05-12      1.531544e-17  6.216478e-34  1.156718e-19  5.081564e-40   
2021-05-13      7.657718e-18  3.108239e-34  5.783588e-20  2.540782e-40   
2021-05-14      3.828859e-18  1.554119e-34  2.891794e-20  1.270391e-40   

Province_State                                                              \
Admin2         Sweetwater         Teton     Uinta Unassigned      Washakie   
2021-05-12       0.000053  1.622131e-27  0.000163  -0.501961  3.157262e-32   
2021-05-13       0.000026  8.110655e-28  0.000081  -0.501961  1.578631e-32   
2021-05-14       0.000013  4.055327e-28  0.000041  -0.501961  7.893154e-33   

Province_State                
Admin2                Weston  
2021-05-12      6.705318e-31  
2021-05-13      3.352659e-31  
2021-05-14      1.676329e-31  

[3 rows x 3336 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN
